# __Data preparation__

This file makes the neccessary preprocessing for the data that will be used in the analysis. 
There is no need to run this file, as the data created by this notebook are enclosed in the repository.

In [1]:
# Importing the required libraries
import pandas as pd 
import geopandas as gpd
import requests
from shapely.geometry import Point

In [2]:
# Load the data which were downloaded from Prague geoportal (https://geoportalpraha.cz/data-a-sluzby/91fda93bea914b2d8c94806edfb90e7f_0)
metro = 'DOP_PID_VSTUPY_B.shp'
metro_ = gpd.read_file(metro)
metro_.head(2)


,UZEL_NAZEV,VEST_NAZEV,VST_POPIS,GLOBALID,VST_LINKA,VST_NAZEV,UZEL_CISLO,VST_KOD,VEST_KOD,VST_MIM_OD,VST_MIM_DO,VST_SCHOD,VST_ESKAL,VST_VYTAH,geometry
0,Invalidovna,Invalidovna,"ul. Sokolovská, sídliště Invalidovna, TRAM cen...",{CE9A5608-9C51-49B3-938B-284838973662},B,E1 ul. Sokolovská,655,1,1,NaN,NaN,0,3,0,POINT (-739593.333 -1042351.545)
1,Kobylisy,Střelničná,"Katastrál. úřad, ul. Pod Sídlištěm, BUS sm. Bo...",{F402DCCA-574D-4BD3-942D-203768BFF85E},C,E8 Katastrální úřad,675,8,1,NaN,NaN,1,0,0,POINT (-739801.330 -1039215.510)


In [3]:
# Drop unnecessary columns and change the crs to WGS84
metro = metro_.drop(columns=['VEST_NAZEV','VST_POPIS', 'GLOBALID',
                              'VST_NAZEV', 'UZEL_CISLO', 'VST_KOD', 'VEST_KOD', 'VST_MIM_OD', 'VST_MIM_DO', 'VST_SCHOD', 'VST_ESKAL', 'VST_VYTAH']).to_crs(epsg=4326)

metro.head(2)

,UZEL_NAZEV,VST_LINKA,geometry
0,Invalidovna,B,POINT (14.46453 50.09729)
1,Kobylisy,C,POINT (14.45569 50.12497)


In [4]:
# Create a function to calculate the centroid for each metro dattion
def calculate_centroid(group):
    centroid = group.unary_union.centroid
    return gpd.GeoDataFrame({'Station Name': [group['UZEL_NAZEV'].iloc[0]], 'Metro Line': [group['VST_LINKA'].iloc[0]],'geometry': [centroid],})

# Apply the function to the metro data
centroids = metro.groupby('UZEL_NAZEV').apply(calculate_centroid)
centroids = centroids.reset_index(drop=True)
centroids = centroids.set_crs(epsg =4326)
centroids.head(2)

,Station Name,Metro Line,geometry
0,Anděl,B,POINT (14.40395 50.06918)
1,Bořislavka,A,POINT (14.36367 50.09842)


In [5]:
# Add information about the coordinates
metro = centroids
metro['Latitude'] = metro['geometry'].y
metro['Longitude'] = metro['geometry'].x
metro['Coordinates'] = metro['Latitude'].astype(str) + ',' + metro['Longitude'].astype(str)
# Add the missing metro line information
metro.loc[metro['Station Name'] == 'Florenc', 'Metro Line'] = 'B,C'
metro.head(2)

,Station Name,Metro Line,geometry,Latitude,Longitude,Coordinates
0,Anděl,B,POINT (14.40395 50.06918),50.069181,14.403950,"50.069181435064294,14.403950258970752"
1,Bořislavka,A,POINT (14.36367 50.09842),50.098418,14.363665,"50.09841763504169,14.363665020922902"


In [6]:
# Change crs back to Krovak
metro_krovak = centroids.to_crs(epsg=5514)
metro_krovak['Latitude'] = metro_krovak['geometry'].y
metro_krovak['Longitude'] = metro_krovak['geometry'].x
metro_krovak['Coordinates'] = metro_krovak['Latitude'].astype(str) + ',' + metro_krovak['Longitude'].astype(str)
metro_krovak.head(2)


,Station Name,Metro Line,geometry,Latitude,Longitude,Coordinates
0,Anděl,B,POINT (-744313.969 -1044858.417),-1.044858e+06,-744313.969031,"-1044858.4166476722,-744313.9690311057"
1,Bořislavka,A,POINT (-746724.950 -1041242.884),-1.041243e+06,-746724.949597,"-1041242.8838896882,-746724.9495974302"


In [7]:
# Safe file for further analysis
metro_krovak.to_file("metro5514.gpkg", driver="GPKG")

In [8]:
# Get data from Foursquare API
url = "https://api.foursquare.com/v3/places/search"
headers = {
    "Accept": "application/json",
    "Authorization": "####"
}
# Create an empty dataframe
df2 = pd.DataFrame()
# Create a list of categories
categorylist = ["Arts and Entertainment", "Business and Professional Services", "Community and Government", "Dining and Drinking", "Event", "Health and Medicine",
                "Landmarks and Outdoors", "Retail", "Sports and Recreation", "Travel and Transportation"
                ]
# Loop through the metro stations and the categories
for i in range(len(metro)):
    for j in range(len(categorylist)):
        params = {
            "query": "value",
            "ll": "value2",
            # Set the radius to 400 meters
            "radius":400,
            "sort":"DISTANCE"
        }
        params['query'] = categorylist[j]
        params['ll'] = metro['Coordinates'][i]
        response = requests.request("GET", url, params=params, headers=headers)
        json = response.json()
        
        # If the response contains the results, create a dataframe and add the station name and the category
        if "results" in json:
            df = pd.DataFrame(json['results'])
            df['Station Name'] = metro['Station Name'][i]
            df['Venue Type'] = categorylist[j]            
            df.rename(columns={"name": "Venue Name", "distance": "Venue Distance"}, inplace=True)
            df2 = pd.concat([df2,df]) 

In [9]:
merged_df= pd.merge(metro, df2, how='left', on='Station Name')
merged_df.head(2)


,Station Name,Metro Line,geometry,Latitude,Longitude,Coordinates,fsq_id,categories,chains,closed_bucket,Venue Distance,geocodes,link,location,Venue Name,related_places,timezone,Venue Type
0,Anděl,B,POINT (14.40395 50.06918),50.069181,14.40395,"50.069181435064294,14.403950258970752",523ccc0b11d263770febf9e3,"[{'id': 10008, 'name': 'Casino', 'short_name':...",[],VeryLikelyOpen,67.0,"{'drop_off': {'latitude': 50.069192, 'longitud...",/v3/places/523ccc0b11d263770febf9e3,"{'country': 'CZ', 'cross_street': '', 'formatt...",Casino Top Games Admiral,{},Europe/Prague,Arts and Entertainment
1,Anděl,B,POINT (14.40395 50.06918),50.069181,14.40395,"50.069181435064294,14.403950258970752",56e2e489498e1b96d65230e7,"[{'id': 10018, 'name': 'Gaming Cafe', 'short_n...",[],Unsure,95.0,"{'main': {'latitude': 50.068907, 'longitude': ...",/v3/places/56e2e489498e1b96d65230e7,"{'address': 'Ostrovskeho 16', 'country': 'CZ',...",Dětský klub Barbucha,{},Europe/Prague,Arts and Entertainment


In [10]:
# Create functions to extract the category name, latitude and longitude
def extract_category_name(categories):
    if isinstance(categories, list) and len(categories) > 0:
        return categories[0].get('name')
    return None
def extract_lat(data):
    latitude = data['main']['latitude']
    return latitude
def extract_long(data):
    longitude = data['main']['longitude']
    return longitude

# Apply the functions to the dataframe
merged_df['Venue Latitude'] = merged_df['geocodes'].apply(extract_lat)
merged_df['Venue Longitude'] = merged_df['geocodes'].apply(extract_long)
merged_df['Venue Coordinates'] = merged_df['Venue Latitude'].astype(str) + ',' + merged_df['Venue Longitude'].astype(str)
merged_df['Venue Subtype'] = merged_df['categories'].apply(extract_category_name)

# Drop the columns where the venue subtype is missing
merged_df = merged_df.dropna(subset=['Venue Subtype'])
merged_df.head(2)

,Station Name,Metro Line,geometry,Latitude,Longitude,Coordinates,fsq_id,categories,chains,closed_bucket,...,link,location,Venue Name,related_places,timezone,Venue Type,Venue Latitude,Venue Longitude,Venue Coordinates,Venue Subtype
0,Anděl,B,POINT (14.40395 50.06918),50.069181,14.40395,"50.069181435064294,14.403950258970752",523ccc0b11d263770febf9e3,"[{'id': 10008, 'name': 'Casino', 'short_name':...",[],VeryLikelyOpen,...,/v3/places/523ccc0b11d263770febf9e3,"{'country': 'CZ', 'cross_street': '', 'formatt...",Casino Top Games Admiral,{},Europe/Prague,Arts and Entertainment,50.069319,14.404959,"50.069319,14.404959",Casino
1,Anděl,B,POINT (14.40395 50.06918),50.069181,14.40395,"50.069181435064294,14.403950258970752",56e2e489498e1b96d65230e7,"[{'id': 10018, 'name': 'Gaming Cafe', 'short_n...",[],Unsure,...,/v3/places/56e2e489498e1b96d65230e7,"{'address': 'Ostrovskeho 16', 'country': 'CZ',...",Dětský klub Barbucha,{},Europe/Prague,Arts and Entertainment,50.068907,14.402680,"50.068907,14.40268",Gaming Cafe


In [11]:
venues_df = merged_df[['Venue Type', 'Venue Subtype','Venue Name', 'Station Name','Metro Line','Venue Latitude', 'Venue Longitude', 'Venue Coordinates']]

venues_df.head(2)

,Venue Type,Venue Subtype,Venue Name,Station Name,Metro Line,Venue Latitude,Venue Longitude,Venue Coordinates
0,Arts and Entertainment,Casino,Casino Top Games Admiral,Anděl,B,50.069319,14.404959,"50.069319,14.404959"
1,Arts and Entertainment,Gaming Cafe,Dětský klub Barbucha,Anděl,B,50.068907,14.402680,"50.068907,14.40268"


In [12]:
# Create a function to create a point geometry
def create_point_geometry(coord_str):
    latitude, longitude = map(float, coord_str.split(','))
    return Point(longitude, latitude)

# Apply the function to the dataframe
venues_df['geometry'] = venues_df['Venue Coordinates'].apply(create_point_geometry)

# Create GeoDataFrame
venues_df = gpd.GeoDataFrame(venues_df, geometry='geometry', crs='EPSG:4326')

# Change the crs 
venues_krovak = venues_df.to_crs(epsg=5514)
venues_krovak['Venue Latitude'] = venues_krovak['geometry'].y
venues_krovak['Venue Longitude'] = venues_krovak['geometry'].x
venues_krovak['Venue Coordinates'] = venues_krovak['Venue Latitude'].astype(str) + ',' + venues_krovak['Venue Longitude'].astype(str)
venues_krovak.head(2)

C:\Users\annak\AppData\Local\Temp\ipykernel_20360\2927332808.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venues_df['geometry'] = venues_df['Venue Coordinates'].apply(create_point_geometry)


,Venue Type,Venue Subtype,Venue Name,Station Name,Metro Line,Venue Latitude,Venue Longitude,Venue Coordinates,geometry
0,Arts and Entertainment,Casino,Casino Top Games Admiral,Anděl,B,-1.044853e+06,-744240.344475,"-1044853.1203531758,-744240.344474776",POINT (-744240.344 -1044853.120)
1,Arts and Entertainment,Gaming Cafe,Dětský klub Barbucha,Anděl,B,-1.044876e+06,-744408.218547,"-1044876.2352441632,-744408.21854658",POINT (-744408.219 -1044876.235)


In [13]:
venues_df.to_file("venues5514.gpkg", driver="GPKG")